In [1]:
import os
from indicnlp import common
from indicnlp.tokenize import indic_tokenize

# Set up resource path
INDIC_NLP_RESOURCES = "D:/A Internship Documents/Project G/indic_nlp_resources"
common.set_resources_path(INDIC_NLP_RESOURCES)

# Tokenize
text = "नेपालको राजधानी काठमाडौं हो, जहाँ धेरै सांस्कृतिक स्थलहरू छन्। विद्यार्थीहरूले पुस्तकहरू पढ्न र अनुसन्धान गर्न मन पराउँछन्।"
tokens = list(indic_tokenize.trivial_tokenize(text, lang='ne'))
print(tokens)

['नेपालको', 'राजधानी', 'काठमाडौं', 'हो', ',', 'जहाँ', 'धेरै', 'सांस्कृतिक', 'स्थलहरू', 'छन्', '।', 'विद्यार्थीहरूले', 'पुस्तकहरू', 'पढ्न', 'र', 'अनुसन्धान', 'गर्न', 'मन', 'पराउँछन्', '।']
